# Run pw. qpAdm Model for all indivdiuals of the early cluster

Goal: Show that this cluster is heterogenous
In other analysis it was already established that there is substantial heterogeneity

In [1]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp
import itertools as it
from time import time

# For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
# Make sure to have the font installed (it is on cluster for Harald)
rcParams['font.sans-serif'] = ['Arial']

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
os.sys.path.append(path)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

compute-a-16-48.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 32
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


### Do qpAdm Function Imports and Definitions

In [14]:
from python.run_qpadm import qpAdm_run, get_meta_ind_table, modifiy_iid_files, set_iids_to_label
from python.plot_qpadm import plot_qpadm, load_pvals

### Parameters for qpAdm_run are set there!
a12 = ["Mota", "Ust_Ishim", "Kostenki14", "GoyetQ116-1", "Vestonice16", "MA1",
           "ElMiron", "Villabruna", "EHG", "CHG", "Natufian",
           "Levant_N"] # List of ancient European populations for right populations
distal = ["Anatolia_N", "WHG", "Steppe_EMBA", 
          "Iran_N", "Morocco_EN.SG"]
print(f"# Anc_Euro: {len(a12)} populations")
print(f"# Distal sources:{len(distal)} populations")

# Anc_Euro: 12 populations
# Distal sources:5 populations


# 1) Modify .ind File

### 1a) Prepare Early Punic Individuals
Load data table with individual assignments - and produce modified .ind file for use in qpadm

In [3]:
### Populations to overwrite. Typically because they have the ".SG" label
vrs = "49.2"
ind_merged = f"./eigenstrat/anc_only.v{vrs}_outgroups.ind"          # What .ind to load
ind_modified = f"./eigenstrat/anc_only.v{vrs}.punic_early_ind.ind"    # Where to save the modified version to

df_ind = pd.read_csv(ind_merged, delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex", "clst"]
print(f"Loaded {len(df_ind)} Individuals")

df_lbs = pd.read_csv("./data/cluster_assignments_punic.v49.2.tsv", sep="\t")  ### Load external Label file (Google docs)

### Reset Clusters
for _, row in df_lbs.iterrows():
    iid = row['iid']
    
    idx = df_ind["iid"] == iid
    if np.sum(idx)<1:
        print(f"Warning, {iid} not found in original .ind file.")
    
    df_ind.loc[idx, "clst"] = row["clst_qpadm"]
    ### Reset All the Individuals in the Target Ind
    #df_ind = set_iids_to_clst(df_ind, iids=iids, clst=g, savepath="")

### Overwrite PunicCentralMedEarly
idx = df_ind["clst"]=="PunicCentralMedEarly"
df_p_cm_early = df_ind[idx].copy()
iids = df_p_cm_early["iid"].values
df_ind = set_iids_to_label(df_ind, iids_overwrite=iids)

df_ind.to_csv(ind_modified, header=False, sep=" ", index=False)
print(f"Saved updated {len(df_ind)} IIDs to: {ind_modified}")

Loaded 1142 Individuals
Warning, I26842 not found in original .ind file.
Overwriting 18 Individuals
Saved updated 1142 IIDs to: ./eigenstrat/anc_only.v49.2.punic_early_ind.ind


In [19]:
iids

array(['I4798', 'I4799', 'I4800', 'I12666', 'I12847', 'I22115', 'I22121',
       'I22122', 'I22117', 'I22118', 'I22232', 'I22096', 'I24678',
       'I24676', 'I24675', 'I24556', 'I22095', 'I18201'], dtype=object)

# 2) Run qpAdm

### Test run for pair of indiviuals

In [12]:
%%time

target = "I4798"  
sources = ["I4799"]  # Morocco_EN.SG, Algeria_IA
rightpops = a12 + ["Morocco_EN.SG", "Iran_N"]

leftpops = [target] + sources
qpAdm_run(leftpops = leftpops, rightpops = rightpops, 
          output_file = ".".join(leftpops), 
          input_folder = "./eigenstrat/", 
          input_file = "anc_only.v49.2",
          input_ind_suff = ".punic_early_ind",
          par_file_folder = "./parfiles/", 
          output_folder = "./output/qpAdm/v49.2/area51/", 
          path_bin_qpAdm = "/n/groups/reich/hringbauer/git/AdmixTools/bin/qpAdm",
          all_snps=False)

Runtime: 64.207009
CPU times: user 5.68 ms, sys: 7.5 ms, total: 13.2 ms
Wall time: 1min 4s


0

# Run all possible pairs

In [20]:
for p in it.combinations(iids, r=2):  ### One run was with r=1
    rightpops = a12 + ["Morocco_EN.SG", "Iran_N"]
    leftpops = list(p)
    
    qpAdm_run(leftpops = leftpops, rightpops = rightpops, 
              output_file = ".".join(leftpops), 
              input_folder = "./eigenstrat/", 
              input_file = "anc_only.v49.2",
              input_ind_suff = ".punic_early_ind",
              par_file_folder = "./parfiles/", 
              output_folder = "./output/qpAdm/v49.2/early_punic_pairs_ind/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/git/AdmixTools/bin/qpAdm",
              all_snps=False)

FileNotFoundError: [Errno 2] No such file or directory: './parfiles/left.I4798.I4799'

# Pick up the p-values

In [15]:
load_pvals("./output/qpAdm/v49.2/area51/I4798.I4799.log")

(0.140049413, ['I4798', 'I4799'])